# Lab 6 - Protein Structure Files and the Protein Data Bank

## Introduction

Proteins are biological molecules made up of sequences of amino acids. 
Amino acids have the general structure shown below.

<center>
    <img src="./images/amino-acid.png" style="align:center">
</center>


The "R" group in the amino acid structure can have varying identities. 
Depending on the nature of the R group, the amino acid can be classified as hydrophilic or hydrophobic, postively, negatively, or neutrally charged, or other categories.
The sequence of amino acids relative to one another is key to determining protein structure as the amino acids interact with each other to form protein secondary structure such as alpha helices or beta sheets.
Proteins also commonly have accompanying small molecules (also called ligands) or nucleic acids (DNA or RNA).

Determining protein structure has been an active area of research for many decades, and is part of a field called [structural biology](https://www.nigms.nih.gov/education/fact-sheets/Pages/structural-biology.aspx).
The three dimensional structure of proteins plays a role in its biological function and determines much about its interaction with other molecules. Traditionally, the three dimensional structure of proteins had to be determined through laborious experimental methods such as [X-Ray crystallography](https://phys.libretexts.org/Courses/University_of_California_Davis/UCD%3A_Biophysics_200A_-_Current_Techniques_in_Biophysics/X-ray_Protein_Crystallography), Nuclear Magnetic Resonance (NMR) spectroscopy, or cryo-EM.

Computational techniques such as [homology modeling](https://en.wikipedia.org/wiki/Homology_modeling) have also been used.
In homology modeling, sequence fragments from known structures were matched to the unknown structure, and a model is iteratively built.

More recently, machine learning has allowed for leaps in protein structure prediction. 
Notably, AlphaFold from Google's DeepMind was released in 2020. 
AlphaFold also very recently (October 31, 2023) announced the release of a model that is able to predict structure of complexes [containing ligands, proteins, and nucleic acids](https://deepmind.google/discover/blog/a-glimpse-of-the-next-generation-of-alphafold/).

The [Protein Data Bank (PDB)](https://www.rcsb.org/) was established in 1971 at Brookhaven National Laboratory as a repository of protein structures. 
It originally started with 7 structures. 
Today it has over 200,000 experimentally determined protein structures and **over 1 million** computed structure models.

## Lab Overview

In this lab, we will be using a Python library called Biopython to read and analyze MMCIF files.
Although machine learning prediction of protein structure is an exciting area, we will not be delving into that topic for this lab.
Instead, the lab focuses on MMCIF files, Biopython, and structure analysis of proteins.
    
The [Biopython library](https://biopython.org/) is a powerful and versatile collection of tools specifically designed to assist biochemists and biologists with computational work. 
Biopython simplifies tasks such as working with DNA and protein sequences, interacting with online databases, and reading molecular format files like mmCIF and PDB. 

In this notebook, we will use Biopython to parse information from an MMCIF file.

## MMCIF Files
The mmCIF (also called PDBx/mmCIF) file format defines the three dimensional structure of a protein and provides information
about the molecule, any ligands, host organism, and experimental method and conditions under which the data was obtained.
The mmCIF file has a [specific format](https://pdb101.rcsb.org/learn/guide-to-understanding-pdb-data/beginner%E2%80%99s-guide-to-pdb-structures-and-the-pdbx-mmcif-format)
that consists of data entries with certain keywords beginning with `_` followed by their values.

The following is a text excerpt from the PDB 101 page about the mmCIF file format:

---
All data items are identified by name, begin with the underscore character and are composed of a category name and an attribute name. 
The category name is separated from the attribute name by a period:

```
_citation.year

```

This combination of category and attribute may be termed an mmCIF token.

Data categories are presented in two styles: key-value and tabular.

In the key-value style, the mmCIF token is followed directly by a corresponding value.  The following example shows the unit cell parameters from entry 4hhb:

```
_cell.entry_id           4HHB
_cell.length_a           63.150
_cell.length_b           83.590
_cell.length_c           53.800
_cell.angle_alpha        90.00
_cell.angle_beta         99.34
_cell.angle_gamma        90.00
_cell.Z_PDB              4
```
----

BioPython has a function called `MMCIF2Dict` that can read this information into a Python Dictionary.
We can then query the dictionary to pull out data we are interested in.



In [ ]:
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

structure_info = MMCIF2Dict("data/1mbn.cif")

The variable `experimental_info` is now a Python dictionary containing data from the CIF file.

The mmCIF file contains a lot of data. 
You can see all of the possible dictionary keys by doing 

```python
experimental_info.keys()
```

Note that there are a large number of possible keys for an MMCIF file.
Each MMCIF file contains a varying number of these keywords.
The following cell prints the data categories present in the dictionary.

In [ ]:
data_categories = { key.split(".")[0] for key in structure_info.keys() }
print(data_categories)

We can use dictionary rpocessing. in Python to print some of the data of interest. 
In the following cell, we print the data associated with the `_cell` category.

In [ ]:
cell_data = { k:v for k,v in structure_info.items() if "_cell." in k}
cell_data

You can browse then on [this webpage](https://mmcif.wwpdb.org/dictionaries/mmcif_pdbx_v50.dic/Groups/index.html) from the World Wide PDB for information about the data categories.

Note that every MMCIF file can have slightly different keys.

In particular, the `_entity` category contains interesting information:

        "Data items in the ENTITY category record details (such as
       chemical composition, name and source) about the molecular
       entities that are present in the crystallographic structure.

       Items in the various ENTITY subcategories provide a full
       chemical description of these molecular entities.

       Entities are of three types:  polymer, non-polymer and water.
       Note that the water category includes only water;  ordered
       solvent such as sulfate ion or acetone would be described as
       individual non-polymer entities."

In the cell below, find information for the `_entity` category:

In [ ]:
entity_data = { k:v for k,v in structure_info.items() if "_entity." in k}
entity_data

We see a few interesting entries in the `_entity` section.
The output tells us that there are four entities in the structure.
Those have the description "MYOGLOBIN", "HYDROXIDE ION", and "PROTOPORPHYRIN IX CONTAINING FE".

There are many more properties present in the dictionary we've read from the MMCIF. 
You can continue to investigate the dictionary in the cells below. 
You might try determining when this structure was published or what species it belongs to (but there are more interesting things in the notebook, so don't spend too long on this).

## Measuring 3D Properties

This structure we are examining is for [myoglobin]().
We can load the three dimensional structure into a Python object using `MMCIFParser` in Biopython.

In our analysis above, we see that the myoglobin structure contains an entity called "PROTOPORPHYRIN IX CONTAINING FE". 

One useful thing about having the three dimensional structure of the protein is that we can analyze the coordinates to see how different parts of the molecule interact.
In myoglobin, the iron atom (Fe) in the protoporphyrin reversibly binds the oxygen $O_2$ molecule to carry oxygen to muscles. 
We can use BioPython to analyze the binding of this heme group to the protein.

<center>
    <img src="./images/Heme.svg" style="align:center">
</center>

The image below shows myoglobin with the heme group (circled).

<center>
    <img src="images/myoglobin_heme_small.png">
</center>


We will want to use Biopython to measure the neighbors of the iron atom in order to measure which residues the
heme group is bound to.
To perform this analysis, we will have to load the protein into a Biopython structure object.
This time, we will import `MMCIFParser` instead of `MMCIF2Dict`.
This will allow us to do measurements with the coordinates.

In [ ]:
from Bio.PDB.MMCIFParser import MMCIFParser

# Create an MMCIFParser object
parser = MMCIFParser(QUIET=True)

# Parse the mmCIF file
structure = parser.get_structure("myoglobin", "data/1mbn.cif")
model = structure[0]

We can visualize the structure interactively in our Jupyter notebook using `nglview`.

In [ ]:
import nglview as nv

view = nv.show_biopython(structure)
view.center("ligand")
view

The `structure` variable is a Biopython structure object.

The Structure object represents the molecules in the MMCIF file and is hierarchical.
It follows the SMCRA (Structure/Model/Chain/Residue/Atom) architecture :

* A structure consists of models
* A model consists of chains
* A chain consists of residues
* A residue consists of atoms

This is the way many structural biologists/bioinformaticians think about structure, and provides a simple but efficient way to deal with structure. 

For our analysis, we want to find the neighbors of the iron atom.
To do this analysis, we will first need to find our iron atom.
We use the `structure.get_atoms()` function to get all of the atoms in the MMCIF.
The variable `atoms` after this step is a special data type called a **generator**.

A generator will load elements of a collection one a time when iterated over.
This means only one atom would be loaded into memory if we looped through the atoms.
Biopython uses a generator when returning atoms because sometimes proteins can be so large that you don't want to load them all into your computer memory at once.
There is one drawback to generators, however, and that is that you can only use generators once (if you wanted to work with atoms again, you would have to do structure.get_atoms() again.

Consider the code below demonstrating some properties of generators.

In [ ]:
atoms = structure.get_atoms()
type(atoms)

In [ ]:
atoms

We can use a for loop to go over the generator.
You can really see how a generator works using the `__next__()` special method in Python.
This will load the "next" value in the generator.
(Execute this cell a few times to see the atom update)

In [ ]:
atoms.__next__()

A loop will go through all of the atoms. 
After you have finished the generator, you get an error if you try to iterate further.

In [ ]:
atoms = structure.get_atoms()

for atom in atoms:
    pass

atoms.__next__()

Once you finish iterating through a generator, you cannot use the same generator again.
You can convert a generator to a list if it's more convenient, but this should be avoided for large data structures.

In [ ]:
atoms = structure.get_atoms()

# get a list of atoms
atom_list = list(atoms)

# Find the iron atom(s) using a list comprehension.
iron_atoms = [ atom for atom in atom_list if atom.element == "FE" ]

print(iron_atoms)

iron_atom = iron_atoms[0]

We know have an iron atom object.
Use the cells below to examine methods and attributes associated with this object.
Does this object follow principles of "Pythonic design" we talked about in our recent discussion session?
What happens if you retrieve chains or residues?

We will now use Biopython to do a neighbor search.
A neighbor search is done in two steps.
First, we create a neighbor search variable using `NeighborSearch(atoms)`.
Then, we do the neighbor search by inputting the coordinates we are looking for neighbors to,
and a cut-off distance for the neighbors.
Any atom that is within the specified cut-off distance is considered to be a neighbor.

In [ ]:
from Bio.PDB import NeighborSearch

# Define the maximum distance for a neighbor.
cutoff_distance = 4

# Create a neighbor search.
neighbor_search = NeighborSearch(atom_list)

neighbors = neighbor_search.search(iron_atom.get_coord(), cutoff_distance)

# Print the number of neighbors here
print(f"The iron atom has {len(neighbors)} neighors.")

We can use a for loop (or alternatively, a comprehension) to get information about through the neighbor atoms that the `search` function found.
We can get the residue of the atom by using `atom.get_parent()`.

In [ ]:
for neighbor in neighbors:
    residue = neighbor.get_parent()
    print(residue.get_resname(), residue.get_id())

The lines that say "HEM" are for atoms that belong to the heme ligand.
We are interested in how the ligand binds to the protein, so we'll modify the for loop to only print neighbors if they are not part of the same residue as the iron.

In [ ]:
iron_residue = iron_atom.get_parent()

for neighbor in neighbors:
    residue = neighbor.get_parent()
    
    if residue != iron_residue:
        print(neighbor.element, residue.get_resname(), residue.get_id()[1])

Our analysis shows us that HIS 93 is a neighbor to the iron in the heme group. 
This histidine is called the "distal histidine" and is known to bind to the iron in the heme group.


You can read more about myoglobin's structure [here](https://sites.chem.utoronto.ca/chemistry/coursenotes/GTM/JM/myoglobin/start.htm).

Try increasing the distance to 5. What do you find?

## Exercise - Repeat analysis for a Zinc Finger

Zinc fingers are small protein motifs that are characterized by the coordination of one or more zinc ions to stabilize their structure, enabling them to interact with other molecules such as DNA, RNA, or proteins.
These motifs are commonly found in transcription factors, where they play a crucial role in the recognition and binding of specific DNA sequences, thereby regulating gene expression.

For this challenge, you will analyze the structure `1a1t`.
This structure is the HIV-1 nucleocapsid protein, which contains two zinc fingers that grip the viral RNA during budding of the virus. 
Use Biopython to determine what protein residues are neighbors to zinc in this structure.

<center>
    <img src="images/1a1t.png" style="height:350px">
</center>

**Hint 1** - The structure `1a1t` is determined by NMR and contains 25 structure models. When you load the structure using the MMCIF parser, you will get a list of 25 structures. You can analyze only one of them by using list indexing.

**Hint 2** - The structure also contains more than one Zinc atom, so if you would like to analyze both that are present in the structure, you will have to use your Python knowledge to slightly modify your code.